In [1]:
import gzip
import numpy as np
import csv
import matplotlib.pyplot as plt
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D

In [3]:
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [ ]:
csvFile = open("instance.csv", "r")
reader = csv.reader(csvFile)

In [84]:
training_set = []
validation_set = []
all_set = []
i = 1
for elem in readGz("/Users/linbinyang/Desktop/course-fall2018/cse258/data/assignment1/train.json.gz"):
    if i <= 100000:
        training_set.append(elem)
        i = i + 1
    elif i <= 200000:
        validation_set.append(elem)
        i = i + 1
    all_set.append(elem)

In [106]:
# We have build our training data and validation data
user = defaultdict(list)
item = defaultdict(list)
U_I_R = []
for elem in training_set:
    unit = []
    item_R = []
    user_R = []
    unit.append(elem['reviewerID'])
    user_R.append(elem['reviewerID'])
    unit.append(elem['itemID'])
    item_R.append(elem['itemID'])
    unit.append(elem['rating'])
    user_R.append(elem['rating'])
    item_R.append(elem['rating'])
    U_I_R.append(unit)
    user[elem['reviewerID']].append(item_R)
    item[elem['itemID']].append(user_R)

In [6]:
def inner_multiply(x, y):
    return sum([x[i]*y[i] for i in range(len(x))])

In [7]:
def ObjectFunction(alpha, beta_u, beta_i, gamma_u, gamma_i, lam, U_I_R, user, item):
    res = 0
    for elem in U_I_R:
        res = res + (alpha + beta_u[elem[0]] + beta_i[elem[1]] + inner_multiply(gamma_u[elem[0]], gamma_i[elem[1]]) - elem[2])**2
    for elem in user:
        res = res + lam*beta_u[elem] * beta_u[elem]
        res = res + lam*sum([unit_gamma_u*unit_gamma_u for unit_gamma_u in gamma_u[elem]])
    for elem in item:
        res = res + lam*beta_i[elem] * beta_i[elem]
        res = res + lam*sum([unit_gamma_i*unit_gamma_i for unit_gamma_i in gamma_i[elem]])
    return res

In [68]:
def ObjectPrime(alpha, beta_u, beta_i, gamma_u, gamma_i, lam, U_I_R, user, item):
    res = []
    d_alpha = 0
    d_beta_u = defaultdict(int)
    d_beta_i = defaultdict(int)
    d_gamma_u = defaultdict(list)
    d_gamma_i = defaultdict(list)
    for elem in U_I_R:
        d_alpha = alpha + 2*(beta_u[elem[0]] + beta_i[elem[1]] + inner_multiply(gamma_u[elem[0]], gamma_i[elem[1]])-elem[2])
    for key_u in beta_u:
        d_beta_u[key_u] = 0
        for item_unit in user[key_u]:
            d_beta_u[key_u] = d_beta_u[key_u] + 2*(alpha + beta_u[key_u] + beta_i[item_unit[0]] + inner_multiply(gamma_u[key_u], gamma_i[item_unit[0]])-item_unit[1])
        d_beta_u[key_u] = d_beta_u[key_u] + 2*lam*beta_u[key_u]
    for key_i in beta_i:
        d_beta_i[key_i] = 0
        for user_unit in item[key_i]:
            d_beta_i[key_i] = d_beta_i[key_i] + 2*(alpha + beta_u[user_unit[0]] + beta_i[key_i] + inner_multiply(gamma_u[user_unit[0]], gamma_i[key_i]) - user_unit[1])
        d_beta_i[key_i] = d_beta_i[key_i] + 2*lam*beta_i[key_i]
    for user_unit in gamma_u:
        unit_gamma_u = [0]*len(gamma_u[user_unit])
        for i in range(len(gamma_u[user_unit])):
            for item_unit in user[user_unit]:
                unit_gamma_u[i] = unit_gamma_u[i] + 2*(alpha+beta_u[user_unit]+beta_i[item_unit[0]]+inner_multiply(gamma_u[user_unit], gamma_i[item_unit[0]])-item_unit[1])*gamma_i[item_unit[0]][i]
            unit_gamma_u[i] = unit_gamma_u[i] + 2*lam*gamma_u[user_unit][i]
        d_gamma_u[user_unit] = unit_gamma_u
    for item_unit in gamma_i:
        unit_gamma_i = [0]*len(gamma_i[item_unit])
        for i in range(len(gamma_i[item_unit])):
            for user_unit in item[item_unit]:
                unit_gamma_i[i] = unit_gamma_i[i] + 2*(alpha+beta_u[user_unit[0]]+beta_i[item_unit]+inner_multiply(gamma_u[user_unit[0]], gamma_i[item_unit])-user_unit[1])*gamma_u[user_unit[0]][i]
            unit_gamma_i[i] = unit_gamma_i[i] + 2*lam*gamma_i[item_unit][i]
        d_gamma_i[item_unit] = unit_gamma_i   
    res.append(d_alpha)
    res.append(d_beta_u)
    res.append(d_beta_i)
    res.append(d_gamma_u)
    res.append(d_gamma_i)
    return res

In [71]:
# update parameters for each iterations
def updateParameter(d_res, lr, alpha, beta_u, beta_i, gamma_u, gamma_i):
    coff = []
    alpha = alpha - lr*d_res[0]
    for key_u in beta_u:
        beta_u[key_u] = beta_u[key_u] - lr*d_res[1][key_u]
    for key_i in beta_i:
        beta_i[key_i] = beta_i[key_i] - lr*d_res[2][key_i]
    for user_unit in gamma_u:
        for i in range(len(gamma_u[user_unit])):
            gamma_u[user_unit][i] = gamma_u[user_unit][i] - lr*d_res[3][user_unit][i]
    for item_unit in gamma_i:
        for i in range(len(gamma_i[item_unit])):
            gamma_i[item_unit][i] = gamma_i[item_unit][i] - lr*d_res[4][item_unit][i]
    coff.append(alpha)
    coff.append(beta_u)
    coff.append(beta_i)
    coff.append(gamma_u)
    coff.append(gamma_i)
    return coff

In [120]:
# initialize parameters
# Initial beta_u and beta_i
coff = []
lam = 6.6
lr = 0.00166
beta_u = defaultdict(int)
beta_i = defaultdict(int)
alpha = 0
gamma_u = defaultdict(list)
gamma_i = defaultdict(list)
for key_u in user:
    beta_u[key_u] = 0
    gamma_u[key_u] = np.random.randn(1).tolist()
for key_i in item:
    beta_i[key_i] = 0
    gamma_i[key_i] = np.random.randn(1).tolist()
coff.append(alpha)
coff.append(beta_u)
coff.append(beta_i)
coff.append(gamma_u)
coff.append(gamma_i)

In [112]:
# training process
def train(iteration_num, coff, lam, lr, U_I_R, user, item):
    res_list = []
    for i in range(iteration_num):
        res = ObjectFunction(coff[0], coff[1], coff[2], coff[3], coff[4], lam, U_I_R, user, item)
        if (res < 100000):
            lr = lr*0.99
            # learning rate decay
        if i%10 == 0:
            print (f"MSE = {res}")
            res_list.append(res)
        d_res = ObjectPrime(coff[0], coff[1], coff[2], coff[3], coff[4], lam, U_I_R, user, item)
        coff = updateParameter(d_res, lr, coff[0], coff[1], coff[2], coff[3], coff[4])
    number_of_samples = range(0, int(iteration_num/10))
    plt.title('Recommendation System on latent factor model')
    plt.xlabel('Number of samples')
    plt.ylabel('MSE error')
    plt.plot(number_of_samples, res_list, color='blue')
    plt.show()
    return coff

In [1]:
# coff = train(800, coff, lam, lr, U_I_R, user, item)

In [122]:
print (coff[0])

4.509273496281763


In [81]:
def computeMSE(coff, validation_set):
    res = []
    real_label = []
    for elem in validation_set:
        if elem['reviewerID'] in coff[1] and elem['itemID'] in coff[2]:
            unit_res = coff[0] + coff[1][elem['reviewerID']] + coff[2][elem['itemID']] + inner_multiply(coff[3][elem['reviewerID']], coff[4][elem['itemID']])
            res.append(unit_res)
            real_label.append(elem['rating'])
    return sum([abs(res[i] - real_label[i])**2 for i in range(len(res))])/len(res)

In [124]:
predictions = open("predictions_Rating4.txt", 'w')
for l in open("/Users/linbinyang/Downloads/assignment1/pairs_Rating.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u in coff[1] and i in coff[2]:
        predictions_n = coff[0] + coff[1][u] + coff[2][i] + inner_multiply(coff[3][u], coff[4][i])
    else:
        predictions_n = coff[0]
    if predictions_n > 5:
        predictions.write(u + '-' + i + ',' + str(5) + '\n')
    else:
        predictions.write(u + '-' + i + ',' + str(predictions_n) + '\n')
predictions.close()

In [123]:
print (computeMSE(coff, validation_set))

1.1752082586198969
